In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.randn(2, 20)
net(X)

tensor([[-0.1411, -0.2009,  0.2431, -0.0155,  0.0243,  0.0522,  0.1686, -0.0505,
         -0.1085,  0.2918],
        [ 0.0332, -0.4703, -0.1104,  0.1455, -0.2381, -0.1313,  0.2649, -0.1697,
          0.0578,  0.1854]], grad_fn=<AddmmBackward0>)

In [3]:
class MLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.lin1 = nn.Linear(20, 256)
        self.lin2 = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.lin2(F.relu(self.lin1(X)))

net = MLP()
X = torch.randn(2, 20)
net(X)

tensor([[ 0.0786, -0.1052, -0.0012, -0.0606,  0.0771, -0.0983,  0.0268,  0.3637,
         -0.0857,  0.1727],
        [ 0.1866, -0.0370,  0.1367, -0.0760,  0.0082,  0.3211,  0.3690,  0.5064,
          0.1180,  0.0875]], grad_fn=<AddmmBackward0>)

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args) -> None:
        super().__init__()
        for idx, block in enumerate(args):
            self._modules[str(idx)] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [6]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.2580,  0.1270,  0.0945, -0.2920,  0.0462, -0.2111, -0.2614, -0.0641,
         -0.2888,  0.0584],
        [-0.2429, -0.0260,  0.1330, -0.0474, -0.3083, -0.1601,  0.1233,  0.1071,
         -0.1929,  0.4073]], grad_fn=<AddmmBackward0>)

In [ ]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.randn((20,20), requires_grad=False)
        self.linear = nn.Linear(20,20)
    
    def forward(self, X):
        X = self.linear(X)
        # 这里可以注意一下，为什么是 X @ self.rand_weight
        # 因为 X 的第一个维度用来表示 batchsize，所以可以理解为，X 的 features 存储为行向量的表示形式
        # 这里可以做一个延伸，nn.Linear 本质上存储的 W 是 (out, in) 的形式，在计算的时候，变成了 X @ W^T 
        X = F.relu(X @ self.rand_weight + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [12]:
net = FixedHiddenMLP()
net(X)

tensor(0.1672, grad_fn=<SumBackward0>)